# 巨潮上市公司年报pdf爬取

## 0. 载入包

In [16]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException

from lxml import etree
import re

import time

import requests
from PyPDF2 import PdfReader
from io import BytesIO

import chardet

import pandas as pd
import numpy as np

from multiprocessing import Process, current_process

## 1. 全局信息

In [2]:
# 处理旧的数据 -- 备份，不要动！
index_url = pd.read_csv('index.txt', delimiter='\t', names=['index_url'])
index_url['stkid'] = index_url['index_url'].apply(lambda x: re.search('stockCode=(\d{6})', x).group(1))
index_url['stkid'] = index_url['stkid'].astype(int)

id_counted = pd.read_csv('getted.txt', delimiter='\t', names=['stkid'])
id_counted = id_counted.assign(counted=1)

df3 = pd.merge(index_url,id_counted,on='stkid',how='left')
df3['counted'] = df3['counted'].fillna(0)

num = pd.read_csv('pdf_url.txt', delimiter='\t', names=['stkid', 'year', 'num', 'pdf_url'])

Green = merged_data = pd.merge(df3, num, on='stkid', how='left')

del index_url, id_counted, df3, num
Green = Green.assign(delete=0)
Green.to_csv('Green.csv', index=False)

In [6]:
# 用Pandas DataFrame读取数据
Green = pd.read_csv('Green2.csv')
Green

,stkid,year,num_x,index_url,pdf_url,delete
0,2,2021,16.0,http://www.cninfo.com.cn/new/disclosure/stock?...,http://static.cninfo.com.cn/finalpage/2022-03-...,0
1,2,2020,13.0,http://www.cninfo.com.cn/new/disclosure/stock?...,http://static.cninfo.com.cn/finalpage/2021-03-...,0
2,6,2021,4.0,http://www.cninfo.com.cn/new/disclosure/stock?...,http://static.cninfo.com.cn/finalpage/2022-04-...,0
3,6,2020,6.0,http://www.cninfo.com.cn/new/disclosure/stock?...,http://static.cninfo.com.cn/finalpage/2021-03-...,0
4,7,2021,21.0,http://www.cninfo.com.cn/new/disclosure/stock?...,http://static.cninfo.com.cn/finalpage/2022-04-...,0
...,...,...,...,...,...,...
5458,605589,2021,NaN,http://www.cninfo.com.cn/new/disclosure/stock?...,NaN,0
5459,605598,2020,NaN,http://www.cninfo.com.cn/new/disclosure/stock?...,NaN,0
5460,605598,2021,NaN,http://www.cninfo.com.cn/new/disclosure/stock?...,NaN,0
5461,605599,2020,NaN,http://www.cninfo.com.cn/new/disclosure/stock?...,NaN,0


In [5]:
# 想要把一些变量弄到json里，后面再搞

sleep = 0.3

# 加载关键词组
with open('keywords.txt', 'r+', encoding='utf-8') as keywords:
	keywords_list:list = keywords.read().split('\n')
pattern = "|".join(keywords_list)

# xpath
reports_list_xpath = '//*[@id="main"]/div[3]/div/div[2]/div/div/div[2]/div[1]/div[2]/div/div/div[3]/table/tbody/tr'

## 3. 爬虫过程封装

### 3.1 selenium对象的新方法

In [21]:
# 对一个selenium.WebElement对象进行xpath解析
def element_to_xpath(webelement, xpath:str):
	elment_html = etree.fromstring(webelement.get_attribute('outerHTML'))
	return elment_html.xpath(xpath)[0]
class webdriver_chrome_xpath(webdriver.Chrome):
	# 通过xpath定位元素
	def xpath_find_element(self, xpath:str):
		return self.find_element(By.XPATH, xpath)
	def xpath_find_elements(self, xpath:str):
		return self.find_elements(By.XPATH, xpath)
	# 检查页面是否正确
	def safe_get(self, url:str, xpath:str):
		while True:
			try:
				self.get(url)
				wait_obj = WebDriverWait(self, 10)
				wait_obj.until(
					EC.presence_of_element_located(
						(By.XPATH, xpath)
					)
				)
				break
			except TimeoutException:
				print('TimeoutException:', url)
				continue

class webdriver_chrome_xpath_cninfo(webdriver_chrome_xpath):
	# 通过公告页面的链接获取pdf链接
	def get_pdf_url(self, page_url:str) -> str:
		pdf_url_xpath = '//*[@id="noticeDetail"]/div/div[2]/div[1]/a'
		# 打开一个新标签页
		self.execute_script("window.open('','_blank');")
		self.switch_to.window(self.window_handles[-1])

		# self.execute_script("document.getElementsByTagName('embed')[0].style.display = 'none';")	# 避免加载pdf预览，直接在初始化的时候写
		# 获取pdf链接
		link = ''
		while link == '':
			self.safe_get(page_url, pdf_url_xpath)
			time.sleep(sleep)
			link = self.xpath_find_element(pdf_url_xpath).get_attribute('href')
		# 关闭临时标签页并返回
		self.close()
		self.switch_to.window(self.window_handles[0])
		return link			

### 3.2 爬虫过程方法

In [8]:
# 根据连接解析pdf，提取关键词
def pdf_keywords_count(link:str, pattern:str) -> int:
	pdf_content = requests.get(link).content
	pdf_file = BytesIO(pdf_content)
	pdf_reader = PdfReader(pdf_file)
	count = 0
	for page in pdf_reader.pages:
		content = page.extract_text()
		count += len(re.findall(pattern, content))
	return str(count)

# 进程任务分配
def task_distribute(dataframe, task_num):
	table_len = len(dataframe)	# 5065
	n = task_num
	len_2 = int(table_len / task_num)	# 1688

	task_list = []
	p = 0
	for i in range(n):
		if i < n - 1:
			task_list.append([p, (i + 1) * len_2])
			p += len_2
		else:
			task_list.append([p, table_len])
	return task_list

## 2. selenium设置

In [9]:
# 创建Chrome参数对象
options = webdriver.ChromeOptions()
# 添加试验性参数
options.add_experimental_option('excludeSwitches', ['enable-automation'])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument('--headless')	# 隐藏浏览器窗口


In [10]:
task_list = task_distribute(Green, 1)
task_list

[[0, 5463]]

In [19]:
# 并发读取pdf_url
def get_keywords_count(dataframe, task_order:int):
	# 创建Chrome浏览器对象并传入参数
	driver = webdriver_chrome_xpath_cninfo(options=options)
	# 执行Chrome开发者协议命令（在加载页面时执行指定的JavaScript代码）
	driver.execute_cdp_cmd(
		'Page.addScriptToEvaluateOnNewDocument',
		{'source': 'Object.defineProperty(navigator, "webdriver", {get: () => undefined})'}
	)
	
	# 爬虫
	for i in range(task_list[task_order][0], task_list[task_order][1]):
		if pd.isnull(dataframe.loc[i, 'pdf_url']):
			url = dataframe.loc[i, 'index_url']
			year = dataframe.loc[i, 'year']
			stkid = dataframe.loc[i, 'stkid']

			driver.safe_get(url, reports_list_xpath)
			'''
			预留编写翻页功能
			'''
			# 从巨潮上市公司主页取得的公告列表
			reports_list:list = driver.xpath_find_elements(reports_list_xpath)
			fail = 1
			# 遍历每一条公告
			for row in reports_list:
				report_title = element_to_xpath(row, './td/div/a/span[1]').text	# 公告标题文本
				# 筛选需要的公告
				if (not re.search(r'(摘要|正文|补充公告|更正前|已取消|英文版|H股|English)', report_title)) & (re.search(str(year) + '(年|)年度报告', report_title) != None):
					report_link_url = 'http://www.cninfo.com.cn' + element_to_xpath(row, './td/div/a').get('href')	# 第一层link，用于进入巨潮的pdf公告阅读界面
					pdf_url = driver.get_pdf_url(report_link_url)

					if re.search("(http).+\.PDF", pdf_url) == None:
						print("wrong:", stkid, year)
						keywords_count = ""
					else:
						keywords_count = pdf_keywords_count(pdf_url, pattern)

					temp = [str(i) for i in [stkid, year, keywords_count, url, pdf_url, 0]]

					dataframe.loc[i, 'pdf_url'] = pdf_url
					dataframe.loc[i, 'num_x'] = keywords_count
					fail = 0
			if fail == 1:
				dataframe.loc[i, 'delete'] = 1
			dataframe.to_csv('Green2.csv', index=False)

In [23]:
get_keywords_count(Green, 0)

KeyboardInterrupt: 